<a href="https://colab.research.google.com/github/weltonrodrigo/first-order-model/blob/master/deepfake_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 💔 Só funciona no Google Chrome💔.
# Para gerar seu deepfake, você só precisa:


1.   Ir no menu **Runtime -> Run all**
![alt text](https://i.ibb.co/GMT693y/run-all.png)
2.   Aguardar uns 30 segundos enquanto roda a inicialização da rede neural.
2.   Fazer o upload do seu vídeo na <h2>🎬 Célula 1</h2>
3.   Fazer o upload da imagem na <h2>📸 Célula 2</h2>
2.   Aguardar o resultado ficar pronto, no <h2>🥌 final da página</h2>.


# Um monte de coisa preparada antes que você não precisa mexer.

In [0]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")
from IPython.display import HTML
from base64 import b64encode

In [0]:
!pip install face_alignment imageio-ffmpeg autocrop

In [0]:
!rm -rf first-order-model
!git clone https://github.com/AliaksandrSiarohin/first-order-model


Cloning into 'first-order-model'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 183 (delta 11), reused 12 (delta 5), pack-reused 161
Receiving objects: 100% (183/183), 58.16 MiB | 37.76 MiB/s, done.
Resolving deltas: 100% (88/88), done.


In [0]:
cd first-order-model

/content/first-order-model/first-order-model


In [0]:
!curl -C - -L https://1drv.ws/u/s!AsP1keFmL5J8gZQO3oDQHOXnDS3_qA?e=RpmJNJ -o vox-cpk.pth.tar

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  695M  100  695M    0     0  26.0M      0  0:00:26  0:00:26 --:--:-- 37.8M


In [0]:
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', checkpoint_path='vox-cpk.pth.tar')

In [0]:
import numpy as np
import pandas as pd
import imageio
from skimage.transform import resize
from argparse import ArgumentParser
from skimage import img_as_ubyte
import os
import subprocess
from multiprocessing import Process
import warnings
import sys
import glob
from tqdm import tqdm
import face_alignment

warnings.filterwarnings("ignore")


def compute_aspect_preserved_bbox(bbox, increase_area):
    left, top, right, bot = bbox
    width = right - left
    height = bot - top
 
    width_increase = max(increase_area, ((1 + 2 * increase_area) * height - width) / (2 * width))
    height_increase = max(increase_area, ((1 + 2 * increase_area) * width - height) / (2 * height))

    left = int(left - width_increase * width)
    top = int(top - height_increase * height)
    right = int(right + width_increase * width)
    bot = int(bot + height_increase * height)

    return (left, top, right, bot)

def extract_bbox(frame, fa):
    bbox = fa.face_detector.detect_from_image(frame[..., ::-1])[0]    
    return bbox

def process_video(video_path, output_filename):
    fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._2D, flip_input=False, device='cpu')
    # video_path = os.path.join(args.in_folder, video_filename)
    reader = imageio.get_reader(video_path)
    tube_bbox = None
    frame_list = []
    for i, frame in enumerate(reader):
        if i == 0:
            # bbox = extract_bbox(resize(frame, (360, 640), preserve_range=True), fa)
            bbox = extract_bbox(frame, fa)
            # print(bbox)
            # bbox = bbox * 3
            print(bbox)
            #left, top, right, bot, _ = bbox
            tube_bbox = bbox[:-1]
            break
        frame_list.append(frame)

    print(tube_bbox)
    tube_bbox = compute_aspect_preserved_bbox(tube_bbox, 0.1)
    # print('increased: ' + ', '.join(tube_bbox))
    left, top, right, bot= tube_bbox
    w = int(right - left)
    h = int(bot - top)
    !ffmpeg -y -loglevel warning -hide_banner -i "{video_path}" -filter:v "crop={w}:{h}:{left}:{top},scale=256:256" -async 1 "{output_filename}"
    # print(subprocess.getoutput(['ffmpeg', '-y', '-loglevel', 'debug', '-hide_banner', '-i', video_path, '-filter:v', f'crop={w}:{h}:{left}:{top},scale=256:256', '-async', '1', output_filename]))

# 🎬 Célula 1 - Vídeo


In [0]:
HTML("""
<h1>Escolha abaixo o seu arquivo de vídeo</h1>
""")

In [0]:
from google.colab import files
uploaded = files.upload()

SOURCE_VIDEO_FILENAME = ""
for name, data in uploaded.items():
  with open(name, 'wb') as f:
    f.write(data)
    print ('saved file', name)
    SOURCE_VIDEO_FILENAME = name

  

# 📸 Célula 2 - imagem a ser animada

In [0]:
HTML("""
<h1>Escolha abaixo a sua imagem a ser animada</h1>
""")

In [0]:
uploaded = files.upload()

SOURCE_IMAGE_FILENAME = ""
for name, data in uploaded.items():
  with open(name, 'wb') as f:
    f.write(data)
    print ('saved file', name)
    SOURCE_IMAGE_FILENAME = name

from autocrop import Cropper
cropper = Cropper(width=256, height=256, face_percent=70)

source_image = cropper.crop(SOURCE_IMAGE_FILENAME)
# plt.imshow(source_image)

# Agora o processamento é feito automático

In [0]:
process_video(SOURCE_VIDEO_FILENAME, 'croppped_driver.mp4')
# from IPython.display import HTML
# from base64 import b64encode
# mp4 = open('croppped_driver.mp4','rb').read()
# data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
# HTML("""
# <video controls>
#       <source src="%s" type="video/mp4">
# </video>
# """ % data_url)

[ 49.266464  142.01978   193.7567    346.39972     0.9999977]
[ 49.266464 142.01978  193.7567   346.39972 ]


In [0]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

# source_image = imageio.imread('/content/gdrive/My Drive/deep/magno.png')
driving_video = imageio.mimread('croppped_driver.mp4')


#Resize image and video to 256x256

source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
    

HTML(display(source_image, driving_video).to_html5_video())

In [0]:
from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

#save resulting video
imageio.mimsave('../generated.mp4', [img_as_ubyte(frame) for frame in predictions])
#video can be downloaded from /content folder

HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 199/199 [00:09<00:00, 21.44it/s]


**Create final video**

In [0]:
!ffmpeg \
-hide_banner \
-y \
-loglevel warning \
-i ../generated.mp4 \
-i croppped_driver.mp4 \
-filter_complex  "[0:v]setpts=1/3*PTS,pad=iw*2:ih[int];[int][1:v]overlay=W/2:0[vid]" \
-map '[vid]' -map 1:a:0  \
-c:v h264 -c:a copy -y complete.mp4

Past duration 0.666985 too large


# RESULTADO

In [0]:
mp4 = open('complete.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video controls>
      <source src="%s" type="video/mp4">
</video>
<br /><h1><a target="_blank" href="%s" download="deep_fake.mp4">➡️Clique aqui para o resultado.⬅️</a><h1>
""" % (data_url, data_url))

# ⬆️⬆️ Clique no link acima ⬆️⬆️ para fazer o download do resultado⬆️⬆️